**CNN-lytical Assignment-3**

**Import libraries here**

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.autograd import Variable

import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random
from sklearn import preprocessing
%matplotlib inline


def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

In [2]:
epochs = 20
batch_size = 50
learning_rate = 0.001
num_workers = 4

**Load Dataset**

In [3]:
# Split into X_train, y_train, X_test, y_test
# you can use stratified splitting from sklearn library

from google.colab import drive

#drive already mounted
drive.mount('/content/drive') 

DATA_PATH = "/content/drive/MyDrive/cs231n/assignments/assignment3/train_cifar.pkl"


with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

X = data['X']
y = data['y']

#split
X = torch.reshape(torch.from_numpy(X),(50000,3,32,32))
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.1) 

X_train = X_train.float()
X_test = X_test.float()

Mounted at /content/drive


In [4]:
transform_train = transforms.Compose([
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([                                 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


In [5]:
# display a 4x4 grid, 
# choose 16 images randomly, display the images as well as corresponding labels


**Creating a Dataset Class**

In [6]:
# define your dataset class

class CIFAR_Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y, transform = transform_train):
        'Initialization'
        #X for images, y for labels
        y = torch.from_numpy(y)
        y = y.squeeze(1)
        self.X = X
        self.y = y
        self.transform = transform

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        image = self.X[index]
        label = self.y[index]
        image = torch.div(image, 255.0)
        return image, label

train_dataset = CIFAR_Dataset(X_train, y_train)
trainset_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)

test_dataset = CIFAR_Dataset(X_test, y_test, transform_test)
testset_loader = torch.utils.data.DataLoader(test_dataset, batch_size = y_test.shape[0], shuffle = False, num_workers = num_workers)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**nn.Module for your model**

In [7]:
# define a child class of nn.Module for your model
# specify the architecture here itself

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=48, kernel_size=(3,3), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=(3,3), padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=(3,3), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=192, out_channels=256, kernel_size=(3,3), padding=(1,1))
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(in_features=8*8*256, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=64)
        self.Dropout = nn.Dropout(0.25)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x)) #32*32*48
        x = F.relu(self.conv2(x)) #32*32*96
        x = self.pool(x) #16*16*96
        x = self.Dropout(x)
        x = F.relu(self.conv3(x)) #16*16*192
        x = F.relu(self.conv4(x)) #16*16*256
        x = self.pool(x) # 8*8*256
        x = self.Dropout(x)
        x = x.view(-1, 8*8*256) # reshape x
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)
        return x

**Prediction & Accuracy**

In [8]:
def predict(model, X_test, y_test):
  test_dataset = CIFAR_Dataset(X_test, y_test, transform_test)
  testset_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers)

  model.eval()
  correct = 0
  total = 0
  # arr = np.empty((50,), int)
  for data in testset_loader:
        images, labels = data
        images = Variable(images).cuda()
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        labels = labels.numpy()
        predicted = predicted.cpu().data.numpy()
        # arr = np.append(arr, predicted)
        correct += (predicted== labels).sum()
        total += labels.size
  # return predicted.numpy()
  print('Accuracy: %f %%' % (100.0 * correct / float(total)))
  # return arr

In [9]:
def accuracy(pred, labels):
  correct = 0
  correct += (pred== labels).sum()
  total = labels.size
  print('Accuracy: %f %%' % (100.0 * correct / float(total)))


**Training loop**

In [10]:
# criterion = nn.CrossEntropyLoss()
def train(model, criterion, trainset_loader, display_step = None):
  #optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
  for epoch in range(epochs):
      running_loss = 0.0

      if epoch <= 10:
          optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      elif epoch > 10 and epoch <= 25:
          optimizer = optim.Adam(model.parameters(), lr=(learning_rate)/10)
      else:
          optimizer = optim.Adam(model.parameters(), lr=(learning_rate)/50)  

      for i, data in enumerate(trainset_loader):
          input_data, labels = data # data is a list of 2, the first element is 4*3*32*32 (4 images) the second element is a list of 4 (classes)
          input_data, labels = Variable(input_data).cuda(),Variable(labels).cuda()

          optimizer.zero_grad() # every time reset the parameter gradients to zero

          # forward backward optimize
          output = model(input_data)
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()

          # print the loss
          running_loss += loss.item()
      # print the loss after every epoch
      print('loss in epoch ' + str(epoch + 1) + ': ' + str(running_loss / y_train.size))  

      #Test for accuracy after every 5 epochs  
      if (epoch + 1)%5 == 0:
          predict(model, X_test, y_test)
          # accuracy(pred, y_test)
      elif epoch == epochs - 1:
          predict(model, X_test, y_test)
          # accuracy(pred, y_test)


**Initialize weights**

In [11]:
def init_weights():
  #On internet articles were saying pyTorch automatically initializes weights for us
  #and those work pretty well, so have postponed the self-initialisation for now
  pass

**Actually training your model**

In [12]:
model = ConvNet()
model.cuda()
criterion = nn.CrossEntropyLoss()
train(model, criterion, trainset_loader)
predict(model, X_train, y_train)
# accuracy(pred_train, y_train)
print()
predict(model, X_test, y_test)
# accuracy(pred_test, y_test)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss in epoch 1: 0.03932684526708391
loss in epoch 2: 0.03370031324227651
loss in epoch 3: 0.031452283612887065
loss in epoch 4: 0.029794972456826104
loss in epoch 5: 0.02835657064517339
Accuracy: 50.220000 %
loss in epoch 6: 0.025762310294310253
loss in epoch 7: 0.02369705669482549
loss in epoch 8: 0.02168633870681127
loss in epoch 9: 0.019221675834390852
loss in epoch 10: 0.016489892703957027
Accuracy: 52.920000 %
loss in epoch 11: 0.0139261148194472
loss in epoch 12: 0.007475222032268842
loss in epoch 13: 0.0056405608953701124
loss in epoch 14: 0.004516532034344143
loss in epoch 15: 0.0035895001195371153
Accuracy: 53.500000 %
loss in epoch 16: 0.0027734548457380797
loss in epoch 17: 0.002069966994681292
loss in epoch 18: 0.0014939992480393913
loss in epoch 19: 0.001020453080214146
loss in epoch 20: 0.0006557459107393191
Accuracy: 52.880000 %
Accuracy: 99.775556 %

Accuracy: 52.880000 %


**Submission**

In [13]:
# from google.colab import files
# torch.save(model, 'ass_3.pt')
# files.download('ass_3.pt') # download the file from the Colab session for submission